# Fee Splitter deployment
This script lays the foundations for autobribe by deploying the fee splitter while leaving the fee distribution unchanged (100% of the fees go to the fee collector).

To run this notebook you need to install the dependencies from the `scripts` poetry group as well
.

## Setup

In [ ]:
import os

# importing scripting dependencies
import boa
import curve_dao
from eth_account import Account
from boa import NetworkEnv, Env


In [ ]:
# use address book for relevant addresses
import tests.integration.address_book as ab

from dotenv import load_dotenv
load_dotenv()

# get env vars
ETHERSCAN_API_KEY = os.getenv("ETHERSCAN_API_KEY")
PINATA_API_KEY = os.getenv("PINATA_API_KEY")
RPC_URL = os.environ.get("RPC_URL")
PRIVATE_KEY = os.environ.get("PRIVATE_KEY")

## Select mode
Production mode cell is intentionally executed first to avoid deploying things by accident. If one runs all the cells sequentially things will be executed in forking mode.

To deploy in production mode, skip the execution of the fork mode cell.

In [ ]:
# production mode (to deploy on ethereum)
deployer = Account.from_key(PRIVATE_KEY)
eth_env = NetworkEnv(RPC_URL)
boa.set_env(eth_env)

# this automatically sets the eoa as the deployer
boa.env.add_account(deployer)

In [ ]:
# fork mode (for testing) 
# we impersonate convex since they definitely have enough to push a vote
CONVEX_VOTERPROXY = "0x989aeb4d175e16225e39e87d0d97a3360524ad80"

forked_env = Env()
boa.set_env(forked_env)

boa.env.fork(RPC_URL)
boa.env.eoa = CONVEX_VOTERPROXY

## Compilation

In [ ]:
# compiling contract (through boa's magic imports)
from contracts.fee_splitter import FeeSplitter

## Deployment

In [ ]:
# only receiver for now is the fee collector
# we expect the current fee collection behavior to remain unchanged
receivers = [(ab.fee_collector, 10_000, False)]
fs = FeeSplitter(ab.crvusd, ab.controller_factory, receivers, ab.dao)

## Vote

In [ ]:
ACTIONS = [
    (ab.controller_factory, "set_fee_receiver", fs)
]
DESCRIPTION = "Set the fee receiver to the fee splitter, to lay the foundation for autobribe, st-crvUSD and rebalancing donations. This vote **is not** about changing the fee distribution (100% of revenues will still go to veCRV holders)."
curve_dao.create_vote(ab.dao, ACTIONS, DESCRIPTION, ETHERSCAN_API_KEY, PINATA_API_KEY)